# Best runs on all tasks

## Import modules

In [6]:
import openml
from openml import tasks, flows, runs
import sklearn
from sklearn import feature_selection
from sklearn.linear_model import LogisticRegression
import pandas as pd
import pprint
from collections import OrderedDict, Counter
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
import re
import random
import numpy as np
from datetime import datetime
import sys
import json
from itertools import combinations
import signal
import importlib
import utils.functions_analyze_runs
importlib.reload(utils.functions_analyze_runs)
importlib.reload(sklearn)
from utils.functions_analyze_runs import get_run_info_lr, get_run_info

# set api key
openml.config.apikey = open('.key', 'r').readline().strip('\n')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sklearn

In [8]:
sklearn.__version__

'0.21.2'

In [9]:
# get all supervised classification tasks
tasks_all = openml.tasks.list_tasks(task_type_id=1, output_format='dataframe', tag = 'OpenML-CC18')
# drop problematic tasks
tasks_all = tasks_all.drop([3573, 146825, 167121, 167124])


## Get OpenML runs for LR flows

In [10]:
all_flows = openml.flows.list_flows()

flow_ids = []
for i in all_flows.items():
    if 'sklearn' in i[1]['name'] and 'LogisticRegression' in i[1]['name']:
        flow_ids.append(i[1]['id'])

In [11]:
# get only LR evaluations
l = list(range(0,1100,100))
u = [j+100 for j in l]

evals = pd.DataFrame()

for l, u in zip(l,u): 
    evals_batch = openml.evaluations.list_evaluations('area_under_roc_curve',
                                            flow= flow_ids[l:u], 
                                            task=list(tasks_all.tid),
                                            output_format='dataframe')
    evals = evals.append(evals_batch)

In [12]:
# rank evaluations
evals['rank'] = evals.groupby('task_id')['value'].rank('first', ascending=False)

# get best evaluations
best_evals = evals.loc[evals['rank'] <= 5]

## Check categorical / numerical / mixed features

In [13]:
# empty list to populate with feature types
types = []

for i in tasks_all.tid:
    print(i, '', end = '')
    
    # get task
    task = openml.tasks.get_task(i)

    # get dataset object 
    data = openml.datasets.get_dataset(task.dataset_id)

    # get relevant info from dataset object
    X, y, categorical_indicator, attribute_names = data.get_data(dataset_format='array',
                                                                target=data.default_target_attribute)
    
    if not any(categorical_indicator):
        types.append((i, 'numeric'))
    elif all(categorical_indicator):
        types.append((i, 'categorical'))
    else:
        types.append((i, 'mixed'))

cat_num = pd.DataFrame(types, columns = ['tid', 'cat_num'])
cat_num = pd.DataFrame(types, columns=['tid', 'cat_num'])

3 6 11 12 14 15 16 18 22 23 28 29 31 32 37 43 45 49 53 219 2074 2079 3021 3022 3481 3549 3560 3902 3903 3904 3913 3917 3918 7592 9910 9946 9952 9957 9960 9964 9971 9976 9977 9978 9981 9985 10093 10101 14952 14954 14965 14969 14970 125920 125922 146195 146800 146817 146819 146820 146821 146822 146824 167119 167120 167125 167140 167141 

In [14]:
cat_num[cat_num.cat_num=='categorical']

,tid,cat_num
0,3,categorical
16,45,categorical
17,49,categorical
26,3560,categorical
48,14952,categorical
55,146195,categorical
60,146821,categorical
66,167140,categorical


In [15]:
# check distribution
cat_num['cat_num'].value_counts()

numeric        45
mixed          15
categorical     8
Name: cat_num, dtype: int64

In [16]:
# check ids of mixed feature tasks
list(cat_num.tid.loc[cat_num.cat_num == 'categorical'])

[3, 45, 49, 3560, 14952, 146195, 146821, 167140]

In [17]:
task_ids = cat_num[cat_num.cat_num != 'mixed'].tid

In [18]:
best_evals.run_id.sample(frac=1)

65086    10485502
11895    10468139
15746    10467432
73172    10553801
73840    10555108
           ...   
64706    10470074
75344    10555951
58242    10467597
66928    10483739
237      10465007
Name: run_id, Length: 332, dtype: int64

In [19]:
len(best_evals.setup_id.unique())

74

In [45]:
LogisticRegression().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'warn',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [49]:
get_run_info_lr(10483739)

{'C': 0.165918932915,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 57,
 'solver': 'liblinear',
 'penalty': 'l2',
 'multi_class': 'warn',
 'l1_ratio': None,
 'random_state': 1}

In [34]:
tasks_all.tid

3              3
6              6
11            11
12            12
14            14
           ...  
167119    167119
167120    167120
167125    167125
167140    167140
167141    167141
Name: tid, Length: 68, dtype: int64

In [40]:
run_ids = best_evals.groupby("setup_id").first().run_id

## Loop over all tasks

In [43]:
# define timeout handler
def handler(signum, frame):
    raise Exception("Timeout!")
    
# Register the signal function handler
signal.signal(signal.SIGALRM, handler)

<function __main__.handler(signum, frame)>

In [53]:
for i in tasks_all.tid:

    # randomly sample a task
    #i = tasks_all.tid.sample().iloc[0] # this samples from all tasks
    #i = task_ids.sample() # sample from numeric or categorical only
    #i = 31

    # get task
    task = openml.tasks.get_task(i)
    
    # get dataset object
    data = openml.datasets.get_dataset(task.dataset_id)

    # get relevant info from dataset object
    X, y, categorical_indicator, attribute_names = data.get_data(dataset_format='array',
                                                                target=data.default_target_attribute)

    # mask with feature types
    cat = categorical_indicator
    num = [not k for k in categorical_indicator]

    # create column transformers
    numeric_transformer = make_pipeline(#SimpleImputer(strategy='median'), 
                                        StandardScaler())

    categorical_transformer = make_pipeline(#SimpleImputer(strategy='most_frequent'),
                                            OneHotEncoder(handle_unknown='ignore'))

    preprocessor = ColumnTransformer(
    transformers=[
    ('num', numeric_transformer, num),
    ('cat', categorical_transformer, cat)])
    
    
    count = 0
    # loop over runs in random order
    # for k in best_evals.run_id.sample(frac=1):
    for k in run_ids:
    
        # set time limit
        signal.alarm(3600)
        
        print('Run', k, 'on task', i)
        print(datetime.now())
        count += 1
        print(count)
        
        try:
            # get params
            params = get_run_info_lr(k)

            # define classifier
            clf = LogisticRegression(**params)

            # pick pipeline according to feature types
            if not any(categorical_indicator):
                pipe = make_pipeline(SimpleImputer(strategy='median'), StandardScaler(), clf)
            elif all(categorical_indicator):
                pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore'), clf)
            else:
                pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), preprocessor, clf)
                
            # run best model on the task
            run = openml.runs.run_model_on_task(pipe, task, avoid_duplicate_runs=True)

            # print feedbackack
            print('Publish openml run...')

            # push tag
            # run.push_tag('best_models')
            # publish the run
            
            run.publish()
            # print feedback
            print('View run online: https://www.openml.org/r/' + str(run.run_id))
            print('Setup', openml.runs.get_run(run.run_id).setup_id)
            print('Flow', openml.runs.get_run(run.run_id).flow_id)
            print(params)
            print()

        except Exception as e:
            print(e)


Run 1983681 on task 3
2020-08-09 15:46:19.950597
1
'l1_ratio'
Run 2039496 on task 3
2020-08-09 15:46:19.959191
2
'l1_ratio'
Run 8858410 on task 3
2020-08-09 15:46:19.968479
3
'l1_ratio'
Run 8858296 on task 3
2020-08-09 15:46:19.979363
4
'l1_ratio'
Run 8858407 on task 3
2020-08-09 15:46:19.989657
5
'l1_ratio'
Run 9201763 on task 3
2020-08-09 15:46:19.999200
6
'l1_ratio'
Run 9201764 on task 3
2020-08-09 15:46:20.011970
7
'l1_ratio'
Run 10154646 on task 3
2020-08-09 15:46:20.022999
8
'l1_ratio'
Run 10228321 on task 3
2020-08-09 15:46:20.034579
9
'l1_ratio'
Run 10228324 on task 3
2020-08-09 15:46:20.050445
10
'l1_ratio'
Run 10228326 on task 3
2020-08-09 15:46:20.063489
11
'l1_ratio'
Run 10228355 on task 3
2020-08-09 15:46:20.073358
12
'l1_ratio'
Run 10228779 on task 3
2020-08-09 15:46:20.087114
13
'l1_ratio'
Run 10228795 on task 3
2020-08-09 15:46:20.103445
14
'l1_ratio'
Run 10228798 on task 3
2020-08-09 15:46:20.114246
15
'l1_ratio'
Run 10229023 on task 3
2020-08-09 15:46:20.124238
16
'l1

Publish openml run...
View run online: https://www.openml.org/r/10557618
Setup 8275903
Flow 18591
{'C': 0.1, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 10000, 'solver': 'lbfgs', 'penalty': 'l2', 'multi_class': 'auto', 'l1_ratio': None, 'random_state': 1}

Run 10465003 on task 6
2020-08-09 15:50:25.856808
31
Publish openml run...
View run online: https://www.openml.org/r/10557619
Setup 8275904
Flow 18591
{'C': 0.01, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 10000, 'solver': 'lbfgs', 'penalty': 'l2', 'multi_class': 'auto', 'l1_ratio': None, 'random_state': 1}

Run 10465005 on task 6
2020-08-09 15:52:03.581982
32
Publish openml run...
View run online: https://www.openml.org/r/10557620
Setup 8275905
Flow 18591
{'C': 1, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 10000, 'solver': 'lbfgs', 'penalty': 'l2', 'multi_class': 'auto', 'l1_ratio': None, 'random_state': 1}

Run 10465007 on task 6
2020-08-09 

/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/

Publish openml run...
View run online: https://www.openml.org/r/10557623
Setup 8275573
Flow 18591
{'C': 1.0, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 100, 'solver': 'liblinear', 'penalty': 'l2', 'multi_class': 'warn', 'l1_ratio': None, 'random_state': 1}

Run 10468424 on task 6
2020-08-09 15:56:40.802710
36


/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/

...

Could not validate run xml by xsd. - XML does not correspond to XSD schema. Error Element '{http://openml.org/openml}value': [facet 'maxLength'] The value has a length of '20490'; this exceeds the allowed maximum length of '2048'.
 on line 146 column 0. Error Element '{http://openml.org/openml}value': '[{&quot;oml-python:serialized_object&quot;: &quot;component_reference&quot;, &quot;value&quot;: {&quot;key&quot;: &quot;num&quot;, &quot;step_name&quot;: &quot;num&quot;, &quot;argument_1&quot;: [true, true, true, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false,

Could not validate run xml by xsd. - XML does not correspond to XSD schema. Error Element '{http://openml.org/openml}value': [facet 'maxLength'] The value has a length of '20490'; this exceeds the allowed maximum length of '2048'.
 on line 146 column 0. Error Element '{http://openml.org/openml}value': '[{&quot;oml-python:serialized_object&quot;: &quot;component_reference&quot;, &quot;value&quot;: {&quot;key&quot;: &quot;num&quot;, &quot;step_name&quot;: &quot;num&quot;, &quot;argument_1&quot;: [true, true, true, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false,

Run 1983681 on task 167140
2020-08-10 15:31:05.553770
1
'l1_ratio'
Run 2039496 on task 167140
2020-08-10 15:31:05.565010
2
'l1_ratio'
Run 8858410 on task 167140
2020-08-10 15:31:05.580683
3
'l1_ratio'
Run 8858296 on task 167140
2020-08-10 15:31:05.595775
4
'l1_ratio'
Run 8858407 on task 167140
2020-08-10 15:31:05.609366
5
'l1_ratio'
Run 9201763 on task 167140
2020-08-10 15:31:05.622919
6
'l1_ratio'
Run 9201764 on task 167140
2020-08-10 15:31:05.635128
7
'l1_ratio'
Run 10154646 on task 167140
2020-08-10 15:31:05.648055
8
'l1_ratio'
Run 10228321 on task 167140
2020-08-10 15:31:05.660249
9
'l1_ratio'
Run 10228324 on task 167140
2020-08-10 15:31:05.674033
10
'l1_ratio'
Run 10228326 on task 167140
2020-08-10 15:31:05.688341
11
'l1_ratio'
Run 10228355 on task 167140
2020-08-10 15:31:05.701913
12
'l1_ratio'
Run 10228779 on task 167140
2020-08-10 15:31:05.721627
13
'l1_ratio'
Run 10228795 on task 167140
2020-08-10 15:31:05.736753
14
'l1_ratio'
Run 10228798 on task 167140
2020-08-10 15:31:05.75

One or more runs of this setup were already performed on the task.
Run 10554933 on task 167140
2020-08-10 15:34:21.973756
72
One or more runs of this setup were already performed on the task.
Run 10555227 on task 167140
2020-08-10 15:34:23.479426
73
One or more runs of this setup were already performed on the task.
Run 10555695 on task 167140
2020-08-10 15:34:24.978773
74
One or more runs of this setup were already performed on the task.
Run 1983681 on task 167141
2020-08-10 15:34:26.708423
1
'l1_ratio'
Run 2039496 on task 167141
2020-08-10 15:34:26.718419
2
'l1_ratio'
Run 8858410 on task 167141
2020-08-10 15:34:26.727842
3
'l1_ratio'
Run 8858296 on task 167141
2020-08-10 15:34:26.739954
4
'l1_ratio'
Run 8858407 on task 167141
2020-08-10 15:34:26.751786
5
'l1_ratio'
Run 9201763 on task 167141
2020-08-10 15:34:26.764079
6
'l1_ratio'
Run 9201764 on task 167141
2020-08-10 15:34:26.775747
7
'l1_ratio'
Run 10154646 on task 167141
2020-08-10 15:34:26.786587
8
'l1_ratio'
Run 10228321 on task 

One or more runs of this setup were already performed on the task.
Run 10553820 on task 167141
2020-08-10 15:35:40.805305
68
One or more runs of this setup were already performed on the task.
Run 10554118 on task 167141
2020-08-10 15:35:42.327190
69
One or more runs of this setup were already performed on the task.
Run 10554393 on task 167141
2020-08-10 15:35:43.867390
70
One or more runs of this setup were already performed on the task.
Run 10554687 on task 167141
2020-08-10 15:35:45.405839
71
One or more runs of this setup were already performed on the task.
Run 10554933 on task 167141
2020-08-10 15:35:46.917859
72
One or more runs of this setup were already performed on the task.
Run 10555227 on task 167141
2020-08-10 15:35:48.495690
73
One or more runs of this setup were already performed on the task.
Run 10555695 on task 167141
2020-08-10 15:35:50.030416
74
One or more runs of this setup were already performed on the task.
